In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, GlobalMaxPooling1D
from sklearn.metrics import classification_report
import pickle

In [4]:
def read_tbl():
    df = pd.read_csv("../../label/data/output/labeled_data.csv")
    return df 

df = read_tbl()

# preprocess the data
texts = df['gist'].astype(str).tolist()
labels = df['label'].astype(int).tolist()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=42)

# Define the neural network
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=50, input_length=maxlen))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

print(classification_report(y_test, y_pred_classes))

Epoch 1/10
15/15 [==============================] - 0s 6ms/step - loss: 0.6919 - accuracy: 0.5083 - val_loss: 0.6915 - val_accuracy: 0.4667
Epoch 2/10
15/15 [==============================] - 0s 1ms/step - loss: 0.6829 - accuracy: 0.6583 - val_loss: 0.6875 - val_accuracy: 0.5333
Epoch 3/10
15/15 [==============================] - 0s 1ms/step - loss: 0.6739 - accuracy: 0.9750 - val_loss: 0.6820 - val_accuracy: 0.8333
Epoch 4/10
15/15 [==============================] - 0s 1ms/step - loss: 0.6598 - accuracy: 0.9750 - val_loss: 0.6745 - val_accuracy: 0.7667
Epoch 5/10
15/15 [==============================] - 0s 1ms/step - loss: 0.6408 - accuracy: 0.9500 - val_loss: 0.6607 - val_accuracy: 0.6667
Epoch 6/10
15/15 [==============================] - 0s 2ms/step - loss: 0.6121 - accuracy: 0.9667 - val_loss: 0.6397 - val_accuracy: 0.6667
Epoch 7/10
15/15 [==============================] - 0s 2ms/step - loss: 0.5730 - accuracy: 0.9750 - val_loss: 0.6078 - val_accuracy: 0.7333
Epoch 8/10
15/15 [==

In [5]:
model.save('../data/output/gist_classification_model.h5')

with open('../data/output/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
